# Solutions with Cation Ordering
Generation of configurational entropy (${S^{config}}$) using combinatorics coupled with $n^{th}$-order Taylor expansion of the non-configurational Gibbs free energy (${G^*}$).  The final expression for the Gibbs free energy of solution is given by $G = -{T}{S^{config}} + {G^*}$.  

This notebook illustrates construction of this problem using SymPy Python module symbolic mathematics and automatic code generation of C-modules.  

Generally, the Taylor expansion of ${G^*}$ is taken to order two (equivalent to regular solution theory) and cation-ordering between symmetrically non-equivalent crystallographic sites is assumed to be non-convergent, i.e. teh random ordering state is not acheived at finite temperature.  Alternately, cation-ordering may be modeled as convergent, inducing a symmetry breaking phase transition at finite temperature, which necessitates Taylor expansion of ${G^*}$ to at least $4^{th}$ order (in ordering parameter) with retention of only even powers of teh ordering variable(s) in the expansion.

In [ ]:
import pandas as pd
import numpy as np
import sympy as sym
sym.init_printing()

# Solution Properties - Structure of the Equations
There are three terms:
- Terms describing standard state contributions
- Terms describing the configurational entropy of solution
- Terms describing the excess enthalpy of solution  

Assumptions:
- There are $c$ components in the system
- There may be more endmember species, $\sigma$, than there are components, thereby allowing for reciprocal solutions
- Cation ordering is permitted, which may be either convergent or non-convergent
- The configurational entropy formulation allows for coupled substitution, e.g. molecular mixing
- The excess enthalpy is described using a Taylor series expansion in compositional and ordering variables.  The order of the expansion is $\nu$. 

### Define temperature and pressure and the universal gas constant units
- $T$ is temperature in $K$
- $P$ is pressure in $bars$
- $R$ is the universal gas constant in $\frac{J}{{K - mol}}$

In [ ]:
T,P,R = sym.symbols('T P R')

# Example for this notebook: pyroxenes
## Components
Endmembers (8 endmembers): $\rm{M2}\rm{M1}\rm{T}_2\rm{O}_6$
1. ${\rm{CaMgS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
2. ${\rm{CaFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
3. ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
4. ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
5. ${\rm{CaFeAlSi}}{{\rm{O}}_{\rm{6}}}$
6. ${\rm{NaAlS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
7. ${\rm{M}}{{\rm{g}}_2}{\rm{S}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
8. ${\rm{CaCrAlSi}}{{\rm{O}}_{\rm{6}}}$  

## Define number of endmember components

In [ ]:
nc = 8

## Independent compositional variables (for pyroxene)

#### There are seven independent compositional variables: 
$r_1$, $r_2$, ... $r_7$, which are taken as equivalent to the mole fractions of ${\rm{CaFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$, ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$, ... ${\rm{CaCrAlSi}}{{\rm{O}}_{\rm{6}}}$. Therefore the mole fraction of ${\rm{CaMgS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$ is calculated as $1-r_1-r_2-r_3-r_4-r_5-r_6-r_7$.

In [ ]:
r1, r2, r3, r4, r5, r6, r7 = sym.symbols('r_1 r_2 r_3 r_4 r_5 r_6 r_7')

#### There are two ordering variables:
- $s_1$, the difference in mole fractions $\rm{Fe}^{3+}$ and $\rm{Al}$ on the ***M1*** site
- $s_2$, the difference in mole fractions $\rm{Fe}^{2+}$ and $\rm{Mg}$ on the ***M2*** site

In [ ]:
ns = 2
s1, s2 = sym.symbols('s_1 s_2')

#### There are 13 site mole fractions:  
$X_{Na}^{M2}$, $X_{Ca}^{M2}$, $X_{Mg}^{M2}$, $X_{{Fe}^{2+}}^{M2}$  
$X_{Mg}^{M1}$, $X_{{Fe}^{2+}}^{M1}$, $X_{Al}^{M1}$, $X_{Cr}^{M1}$, $X_{{Fe}^{3+}}^{M1}$, $X_{Ti}^{M1}$  
$X_{Al}^{Tet}$, $X_{{Fe}^{3+}}^{Tet}$, $X_{Si}^{Tet}$

In [ ]:
xM2_Mg, xM2_Fe2, xM2_Ca, xM2_Na = sym.symbols('xM2_Mg xM2_Fe2 xM2_Ca xM2_Na')
xM1_Mg, xM1_Fe2, xM1_Al, xM1_Cr, xM1_Fe3, xM1_Ti = sym.symbols('xM1_Mg xM1_Fe2 xM1_Al xM1_Cr xM1_Fe3 xM1_Ti')
xT_Al, xT_Fe3, xT_Si = sym.symbols('xT_Al xT_Fe3 xT_Si')

## Define configurational entropy and configurational Gibbs free energy
X,Y are symbols used in testing

In [ ]:
X = sym.Symbol('X', positive=True)
Y = sym.Symbol('Y', positive=True)

### Configurational entropy - random mixing
If there are two cations on site *s*, and their mole fractions on that site are denoted *X* and *Y*, and if there are *c* such sites in the formula unit, then the number of configurations, $\Omega$, associted with ***random*** mixing of cations on that site is:    
$\Omega  = \left[ {\frac{{\left( {X + Y} \right)!}}{{X!Y!}}} \right]$  
and the molar configurational entropy conribution associated with these configurations is given by Boltzmann's law: ${{\hat S}^{conf}} =  R\log \Omega$:  
${{\hat S}^{conf}} =  cR\log \left[ {\frac{{\left( {X + Y} \right)!}}{{X!Y!}}} \right]$  
Using Stirlings approximation for large factorials, $\log X! = X\log X - X$, the configurational entropy can be written:  
${{\hat S}^{conf}} =  cR\left[ - {X\log X - Y\log Y + \left( {X + Y} \right)\log \left( {X + Y} \right)} \right]$  
  
The following function implements this calculation:

In [ ]:
def Sconf_2cats_random(X, Y):
    A = X*sym.log(X) - X
    B = Y*sym.log(Y) - Y
    ApB = (X+Y)*sym.log(X+Y) - (X+Y)
    return ApB -A - B

Examples:

In [ ]:
print ("Case X, Y:   ", R*Sconf_2cats_random(X, Y))
print ("Case X, 1-X: ", R*Sconf_2cats_random(X,1-X))

### Multiple cations - random mixing

In [ ]:
def Sconf_3cats_random(X, Y, Z):
    if X != 0:
        A = X*sym.log(X) - X
    else:
        A = 0
    if Y != 0:
        B = Y*sym.log(Y) - Y
    else:
        B = 0
    if Z != 0:
        C = Z*sym.log(Z) - Z
    else:
        C = 0
    sum = (X+Y+Z)*sym.log(X+Y+Z) - (X+Y+Z)
    return sum - A - B - C

### Pyroxene entropy contribution from the tetrahedral sites 
Sack and Ghiorso (1994a) assume that Si, Al, Fe3+ mix randomly on the tetrahedral sites.  
Note that $X_{Si}^{Tet}$ on the tetrahedral site is redundant, and appears implicitely as $1 - X_{Si}^{Tet} = X_{Al}^{Tet} + X_{F{e^{3 + }}}^{Tet}$:

In [ ]:
Sconfig_tet = 2*R*Sconf_3cats_random(xT_Al, xT_Fe3, 0).subs(xT_Al+xT_Fe3,1-xT_Si)
Sconfig_tet

In [ ]:
def Sconf_4cats_random(X, Y, Z, W):
    if X != 0:
        A = X*sym.log(X) - X
    else:
        A = 0
    if Y != 0:
        B = Y*sym.log(Y) - Y
    else:
        B = 0
    if Z != 0:
        C = Z*sym.log(Z) - Z
    else:
        C = 0
    if W != 0:
        D = W*sym.log(W) - W
    else:
        D = 0
    sum = (X+Y+Z+W)*sym.log(X+Y+Z+W) - (X+Y+Z+W)
    return sum - A - B - C - D

### Pyroxene entropy contribution from the M2 site
Sack and Ghiorso (1994a) assume that Ca, Mg, Fe mix randomly on the M2 site.  
Note that $X_{Na}^{M2}$ on the 2 site is redundant and appears implicitely as $1 - X_{Na}^{M2} = X_{Ca}^{M2} + X_{Mg}^{M2} + X_{F{e^{2 + }}}^{M2}$:

In [ ]:
Sconfig_M2 = R*Sconf_4cats_random(xM2_Mg, xM2_Fe2, xM2_Ca, 0).subs(xM2_Mg+xM2_Fe2+xM2_Ca, 1-xM2_Na)
Sconfig_M2

### Configurational entropy - coupled substituion
If *X* and *Y* are site mole fractions and there are *c* such sites in the formula unit, then assuming that substition of *X* and *Y* are coupled, the numbet of configurations, $\Omega$, associted with mixing cations on that site is:  
$\Omega  = {\left[ {\frac{{X!}}{{Y!\left( {X - Y} \right)!}}} \right]^c}$  
and the configurational entropy conribution associated with these configurations is:  
${{\hat S}^{conf}} =  cR\log \left[ {\frac{{X!}}{{Y!\left( {X - Y} \right)!}}} \right]$  
Using Stirlings approximation we have:  
${{\hat S}^{conf}} =  cR\left[ {X\log X - Y\log Y - \left( {X - Y} \right)\log \left( {X - Y} \right)} \right]$  
  
The following function implements this calculation:

In [ ]:
def Sconf_2cats_coupled(X, Y):
    A = X*sym.log(X) - X
    B = Y*sym.log(Y) - Y
    if A != B:
        AmB = (X-Y)*sym.log(X-Y) - (X-Y)
    else:
        AmB = 0
    return A - B - AmB

Examples:

In [ ]:
print ("Case X, Y:       ", R*Sconf_2cats_coupled(X, Y))
print ("Case X, 1-X:     ", R*Sconf_2cats_coupled(X, 1-X))
print ("Case X/2, X/2:   ", R*Sconf_2cats_coupled(X/2, X/2))
print ("Case 1-X/2, X/2: ", R*Sconf_2cats_coupled(1-X/2, X/2))

Example: Diopside - Alumino-buffonite join

In [ ]:
R*Sconf_2cats_coupled(xM1_Mg, xM1_Ti).subs([[r1,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s2,0]])

Example: Diopside - Alumino-buffonite - CaTs

In [ ]:
R*(Sconf_2cats_coupled(xM1_Mg+xM1_Al, xM1_Ti)+Sconf_4cats_random(xM1_Mg-xM1_Ti, xM1_Al, 0, 0))

Example: Add Fe2+ to the above

In [ ]:
R*(Sconf_2cats_coupled(xM1_Mg+xM1_Fe2+xM1_Al, xM1_Ti)+Sconf_4cats_random(xM1_Mg+xM1_Fe2-xM1_Ti, xM1_Al, 0, 0))

### Pyroxene entropy contribution from the M1 site 

$X_{Mg}^{M1}$ + $X_{{Fe}^{2+}}^{M1}$ + $X_{Al}^{M1}$ + $X_{Cr}^{M1}$ + $X_{{Fe}^{3+}}^{M1}$ + $X_{Ti}^{M1}$

In [ ]:
Sconfig_M1 = R*(Sconf_2cats_coupled(xM1_Mg+xM1_Fe2+xM1_Al+xM1_Fe3+xM1_Cr, xM1_Ti) + \
   Sconf_4cats_random(xM1_Mg+xM1_Fe2-xM1_Ti, xM1_Al, xM1_Fe3, xM1_Cr) + \
   Sconf_4cats_random(xM1_Mg, xM1_Fe2, 0, 0) \
  ).subs(xM1_Mg+xM1_Fe2+xM1_Al+xM1_Fe3+xM1_Cr,1-xM1_Ti)
Sconfig_M1

### Pyroxene entropy contribution from the M1 site, entropy reduction due to Na on M2 site
Sack and Ghiorso (1994a) model this assuming all Na-endmembers mix as molecular species.  Na on M2 is only associated with Al, Fe3+, Cr and Ti on M1 site.  There is an inconsistency here involving Ti, as Na on M2 with Ti on M1 requires either Al or Fe3+ on the tetrahedral site, and the entriopy on that site is not reduced. 

In [ ]:
Sconfig_Na = R*Sconf_2cats_coupled(xM1_Al+xM1_Fe3+xM1_Cr+xM1_Ti, xM2_Na)
Sconfig_Na

## Molar configurational entropy (pyroxene - from sites)

In [ ]:
S_config = Sconfig_M2 + Sconfig_M1 + Sconfig_tet + Sconfig_Na
S_config

## Solve for site mole fractions (for pyroxene):  
#### Mass balance constraints:
- $r_1$ = $X_{{Fe}^{2+}}^{M2}$ + $X_{{Fe}^{2+}}^{M1}$, (${\rm{CaFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$)
- $r_2$ = ($X_{Al}^{M1}$-$X_{{Fe}^{3+}}^{M1}$-$X_{Na}^{M2}$)/2 + ($X_{Al}^{Tet}$-$X_{{Fe}^{3+}}^{Tet}$-$X_{Cr}^{M1}$) + $X_{Ti}^{M1}$, (${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$) 
- $r_3$ = ($X_{{Fe}^{3+}}^{M1}$-$X_{Al}^{M1}$+$X_{Na}^{M2}$)/2 + ($X_{{Fe}^{3+}}^{Tet}$-$X_{Al}^{Tet}$+$X_{Cr}^{M1}$) + $X_{Ti}^{M1}$, (${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$)  
- $r_4$ = ($X_{Al}^{M1}$+$X_{{Fe}^{3+}}^{M1}$-$X_{Na}^{M2}$)/2 + ($X_{Al}^{Tet}$+$X_{{Fe}^{3+}}^{Tet}$-$X_{Cr}^{M1}$) - $X_{Ti}^{M1}$, (${\rm{CaFeAlSi}}{{\rm{O}}_{\rm{6}}}$)  
- $r_5$ = $X_{Na}^{M2}$, (${\rm{NaAlS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$)  
- $r_6$ = 1 - $X_{Na}^{M2}$ - $X_{Ca}^{M2}$, (${\rm{M}}{{\rm{g}}_2}{\rm{S}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$)  
- $r_7$ = $X_{Cr}^{M1}$, (${\rm{CaCrAlSi}}{{\rm{O}}_{\rm{6}}}$)  

#### Site balance constraints
- $X_{Na}^{M2}$ + $X_{Ca}^{M2}$ + $X_{Mg}^{M2}$ + $X_{{Fe}^{2+}}^{M2}$ = 1  
- $X_{Mg}^{M1}$ + $X_{{Fe}^{2+}}^{M1}$ + $X_{Al}^{M1}$ + $X_{Cr}^{M1}$ + $X_{{Fe}^{3+}}^{M1}$ + $X_{Ti}^{M1}$ = 1  
- $X_{Al}^{Tet}$ + $X_{{Fe}^{3+}}^{Tet}$ + $X_{Si}^{Tet}$ = 1  

#### Charge balance constraints 
- $X_{Na}^{M2}$ + 2($X_{Ca}^{M2}$ + $X_{Mg}^{M2}$ + $X_{{Fe}^{2+}}^{M2}$ + $X_{Mg}^{M1}$ + $X_{{Fe}^{2+}}^{M1}$) + 3($X_{Al}^{M1}$ + $X_{Cr}^{M1}$ + $X_{{Fe}^{3+}}^{M1}$ + 2$X_{Al}^{Tet}$ + 2$X_{{Fe}^{3+}}^{Tet}$) + 4($X_{Ti}^{M1}$+2$X_{Si}^{Tet}$) = 12  
(6 oxygens with a charge of -2; recall that there are 2 tetrahedral sites per formula unit):

In [ ]:
system = [ \
    xM2_Fe2 + xM1_Fe2 - r1, \
    (xM1_Al-xM1_Fe3-xM2_Na)/2 + (xT_Al-xT_Fe3-xM1_Cr/2) + xM1_Ti - r2, \
    (xM1_Fe3-xM1_Al+xM2_Na)/2 + (xT_Fe3-xT_Al+xM1_Cr/2) + xM1_Ti - r3, \
    (xM1_Al+xM1_Fe3-xM2_Na)/2 + (xT_Al+xT_Fe3-xM1_Cr/2) - xM1_Ti - r4, \
    xM2_Na - r5, \
    1 - xM2_Na - xM2_Ca - r6, \
    xM1_Cr - r7, \
    xM1_Fe3 - xM1_Al - s1, \
    xM2_Fe2 - xM2_Mg - s2, \
    xM2_Mg + xM2_Fe2 + xM2_Ca + xM2_Na - 1, \
    xM1_Mg + xM1_Fe2 + xM1_Al + xM1_Cr + xM1_Fe3 + xM1_Ti - 1, \
    xT_Al + xT_Fe3 + xT_Si - 1, \
    xM2_Na + 2*(xM2_Ca+xM2_Mg+xM2_Fe2+xM1_Mg+xM1_Fe2) + 3*(xM1_Fe3+xM1_Al+xM1_Cr+2*xT_Fe3+2*xT_Al) + 4*(xM1_Ti+2*xT_Si) - 12 
]

In [ ]:
ans = sym.linsolve(system, xM2_Mg, xM2_Fe2, xM2_Ca, xM2_Na, xM1_Mg, xM1_Fe2, xM1_Al, xM1_Cr, xM1_Fe3, xM1_Ti, xT_Al, xT_Fe3, xT_Si)
ans

In [ ]:
replace = [[xM2_Mg,ans.args[0][0]], [xM2_Fe2,ans.args[0][1]], [xM2_Ca,ans.args[0][2]], [xM2_Na,ans.args[0][3]], \
           [xM1_Mg,ans.args[0][4]], [xM1_Fe2,ans.args[0][5]], [xM1_Al,ans.args[0][6]], [xM1_Cr,ans.args[0][7]], \
           [xM1_Fe3,ans.args[0][8]], [xM1_Ti,ans.args[0][9]], [xT_Al,ans.args[0][10]], [xT_Fe3,ans.args[0][11]], \
           [xT_Si,ans.args[0][12]]]
replace

## Molar configurational Gibbs free energy (pyroxene - from sites)

In [ ]:
G_config = -T*S_config.subs(replace).simplify()
G_config

# $G^*$ - Non-configurational Gibbs free energy

## Define primary extensive compositional variables
= $n$ is a vector of mole numbers of each component
- $n_T$ is the total number of moles in the solution
- $X$ is a vector of mole fractions of components in the system

In [ ]:
component_string = ''
for i in range(1,nc+1):
    component_string += 'n' + str(i) + ' '
n = sym.Matrix(list(sym.symbols(component_string)))
nT = (sym.ones(1,nc) * n)[0]
X = n/nT
nT, X

## Map component mole fractions to $r$ vector for later substitution

In [ ]:
r_mapping = [[r1,X[1]], [r2,X[2]], [r3,X[3]], [r4,X[4]], [r5,X[5]], [r6,X[6]], [r7,X[7]]]

## Taylor expansion of $G^*$
For a second order expansion, the number of Taylor expansion coefficients is:
- 1 for $G_{0}$
- nc-1 for $G_{r_i}$, $i=1...nc-1$
- ns for $G_{s_i}$, $i=1...ns$
- (nc-1)(nc-2)/2 for $G_{{r_i},{r_{i+1}}}$, $i= 1...nc-2$
- ns(ns-1)/2 for $G_{{s_i},{s_{i+1}}}$, $i= 1...ns-1$
- ns(nc-1) for $G_{{r_i},{s_j}}$, $i= 1...nc-1$, $j=1...ns$
- nc-1 for $G_{{r_i},{r_i}}$, $i= 1...nc-1$
- ns for $G_{{s_i},{s_i}}$, $i= 1...ns$

In [ ]:
var = [r1, r2, r3, r4, r5, r6, r7, s1, s2]
taylor_coeff = [sym.symbols('G0')]
taylor_terms = [sym.S.One]
taylor = taylor_coeff[sym.S.Zero]
count = 1
for i in range(1,nc):
    taylor_coeff.append(sym.symbols('Gr'+str(i)))
    taylor += taylor_coeff[count]*var[i-1]
    taylor_terms.append(var[i-1])
    count += 1
for i in range(1,ns+1):
    taylor_coeff.append(sym.symbols('Gs'+str(i)))
    taylor += taylor_coeff[count]*var[nc-2+i]
    taylor_terms.append(var[nc-2+i])
    count += 1
for i in range(1,nc):
    for j in range(i,nc):
        taylor_coeff.append(sym.symbols('Grr'+str(i)+str(j)))
        taylor += taylor_coeff[count]*var[i-1]*var[j-1]
        taylor_terms.append(var[i-1]*var[j-1])
        count += 1
    for j in range(1,ns+1):
        taylor_coeff.append(sym.symbols('Grs'+str(i)+str(j)))
        taylor += taylor_coeff[count]*var[i-1]*var[nc-2+j]
        taylor_terms.append(var[i-1]*var[nc-2+j])
        count += 1
for i in range(1,ns+1):
    for j in range(i,ns+1):
        taylor_coeff.append(sym.symbols('Gss'+str(i)+str(j)))
        taylor += taylor_coeff[count]*var[nc-2+i]*var[nc-2+j]
        taylor_terms.append(var[nc-2+i]*var[nc-2+j])
        count += 1
print ('Number of Taylor expansion terms = {0:.0f}'.format(count))
taylor

### Endmembers in terms of Taylor expansion coefficients
1. ${\rm{CaMgS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
2. ${\rm{CaFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
3. ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
4. ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
5. ${\rm{CaFeAlSi}}{{\rm{O}}_{\rm{6}}}$
6. ${\rm{NaAlS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
7. ${\rm{M}}{{\rm{g}}_2}{\rm{S}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
8. ${\rm{CaCrAlSi}}{{\rm{O}}_{\rm{6}}}$  

### There are more endmember species than components:
(components are numbered, dependent species are listed below, the list is not exhaustive)  

1. ${\rm{CaMgS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
2. ${\rm{CaFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
3. ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
  1. ${\rm{Mg}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
  2. ${\rm{Fe}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$ 
  3. ${\rm{Ca}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
  4. ${\rm{Mg}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
  5. ${\rm{Fe}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{AlSi}}{{\rm{O}}_{\rm{6}}}$
4. ${\rm{Ca}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
  1. ${\rm{Mg}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
  2. ${\rm{Fe}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
  3. ${\rm{Ca}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
  4. ${\rm{Mg}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
  5. ${\rm{Fe}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{FeSi}}{{\rm{O}}_{\rm{6}}}$
5. ${\rm{CaFeAlSi}}{{\rm{O}}_{\rm{6}}}$
  1. ${\rm{MgFeAlSi}}{{\rm{O}}_{\rm{6}}}$
  2. ${\rm{FeFeAlSi}}{{\rm{O}}_{\rm{6}}}$
  3. ${\rm{CaAlFeSi}}{{\rm{O}}_{\rm{6}}}$
  4. ${\rm{MgAlFeSi}}{{\rm{O}}_{\rm{6}}}$
  5. ${\rm{FeAlFeSi}}{{\rm{O}}_{\rm{6}}}$
  6. ${\rm{CaAl_2Si}}{{\rm{O}}_{\rm{6}}}$
  7. ${\rm{MgAl_2Si}}{{\rm{O}}_{\rm{6}}}$
  8. ${\rm{FeAl_2Si}}{{\rm{O}}_{\rm{6}}}$
  9. ${\rm{CaFe_2Si}}{{\rm{O}}_{\rm{6}}}$
  10. ${\rm{MgFe_2Si}}{{\rm{O}}_{\rm{6}}}$
  11. ${\rm{FeFe_2Si}}{{\rm{O}}_{\rm{6}}}$
6. ${\rm{NaAlS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
  1. ${\rm{NaFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
  2. ${\rm{NaCrS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
  3. ${\rm{NaTiAlSi}}{{\rm{O}}_{\rm{6}}}$
  4. ${\rm{NaTiFeSi}}{{\rm{O}}_{\rm{6}}}$
  5. ${\rm{Na}}\left( {{\rm{M}}{{\rm{g}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{Si_2}}{{\rm{O}}_{\rm{6}}}$
  6. ${\rm{Na}}\left( {{\rm{F}}{{\rm{e}}_{{\rm{1/2}}}}{\rm{T}}{{\rm{i}}_{{\rm{1/2}}}}} \right){\rm{Si_2}}{{\rm{O}}_{\rm{6}}}$
7. ${\rm{M}}{{\rm{g}}_2}{\rm{S}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$
  1. ${\rm{Fe}_2}{\rm{Si}_2}{{\rm{O}}_{\rm{6}}}$
  2. ${\rm{MgFe}}{\rm{Si}_2}{{\rm{O}}_{\rm{6}}}$
  3. ${\rm{FeMg}}{\rm{Si}_2}{{\rm{O}}_{\rm{6}}}$
8. ${\rm{CaCrAlSi}}{{\rm{O}}_{\rm{6}}}$
  1. ${\rm{MgCrAlSi}}{{\rm{O}}_{\rm{6}}}$
  2. ${\rm{FeCrAlSi}}{{\rm{O}}_{\rm{6}}}$
  3. ${\rm{CaCrFeSi}}{{\rm{O}}_{\rm{6}}}$
  4. ${\rm{MgCrFeSi}}{{\rm{O}}_{\rm{6}}}$
  5. ${\rm{FeCrFeSi}}{{\rm{O}}_{\rm{6}}}$

In [ ]:
endmembers = [ \
              [[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]], \
              [[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]], \
              [[r1,0],[r2,1],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]], \
              [[r1,0],[r2,0],[r3,1],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]], \
              [[r1,0],[r2,0],[r3,0],[r4,1],[r5,0],[r6,0],[r7,0],[s1, 1],[s2, 0]], \
              [[r1,0],[r2,0],[r3,0],[r4,0],[r5,1],[r6,0],[r7,0],[s1,-1],[s2, 0]], \
              [[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2,-1]], \
              [[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,1],[s1, 0],[s2, 0]] \
             ]
species = [ \
    [[r1,  0],[r2, 1],[r3, 0],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2,-1]], # Al-Bf + En - Di \
    [[r1,  1],[r2, 1],[r3, 0],[r4, 0],[r5, 0],[r6,1/2],[r7,  0],[s1, 0],[s2, 1]], # Al-Bf + Hd + 1/2En - 2 Di \
    [[r1,1/2],[r2, 1],[r3, 0],[r4, 0],[r5, 0],[r6,  0],[r7,  0],[s1, 0],[s2, 0]], # Al-Bf + 1/2 Hd - 1/2 Di \
    [[r1,1/2],[r2, 1],[r3, 0],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2,-1]], # Al-Bf + En + 1/2 Hd - 3/2 Di \
    [[r1,3/2],[r2, 1],[r3, 0],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2,-1]], # Al-Bf + 3/2 Hd + En - 5/2 Di \
    [[r1,  0],[r2, 0],[r3, 1],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2,-1]], # Bf + En - Di \
    [[r1,  1],[r2, 0],[r3, 1],[r4, 0],[r5, 0],[r6,1/2],[r7,  0],[s1, 0],[s2, 1]], # Bf + Hd + 1/2 En - 2 Di \
    [[r1,1/2],[r2, 0],[r3, 1],[r4, 0],[r5, 0],[r6,  0],[r7,  0],[s1, 0],[s2, 0]], # Bf + 1/2 Hd - 1/2 Di \
    [[r1,1/2],[r2, 0],[r3, 1],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2,-1]], # Bf + En + 1/2 Hd - 3/2 Di \
    [[r1,3/2],[r2, 0],[r3, 1],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2,-1]], # Bf + 3/2 Hd + En - 5/2 Di \
    [[r1,  0],[r2, 0],[r3, 0],[r4, 1],[r5, 0],[r6,  1],[r7,  0],[s1, 1],[s2,-1]], # Es + En - Di \
    [[r1,  1],[r2, 0],[r3, 0],[r4, 1],[r5, 0],[r6,1/2],[r7,  0],[s1, 1],[s2, 1]], # Es + Hd + 1/2 En - 2 Di \
    [[r1,  0],[r2, 0],[r3, 0],[r4, 1],[r5, 0],[r6,  0],[r7,  0],[s1,-1],[s2, 0]], # Es^ \
    [[r1,  0],[r2, 0],[r3, 0],[r4, 1],[r5, 0],[r6,  1],[r7,  0],[s1,-1],[s2,-1]], # Es^ + En - Di \
    [[r1,  1],[r2, 0],[r3, 0],[r4, 1],[r5, 0],[r6,1/2],[r7,  0],[s1,-1],[s2, 1]], # Es^ + Hd + 1/2 En - 2 Di \
    [[r1,  0],[r2, 1],[r3,-1],[r4, 1],[r5, 0],[r6,  0],[r7,  0],[s1,-1],[s2, 0]], # Es^ + Al-Bf - Bf (CaTs) \
    [[r1,  0],[r2, 1],[r3,-1],[r4, 1],[r5, 0],[r6,  1],[r7,  0],[s1,-1],[s2,-1]], # Es^ + Al-Bf - Bf + En - Di (MaTs) \
    [[r1,  1],[r2, 1],[r3,-1],[r4, 1],[r5, 0],[r6,1/2],[r7,  0],[s1,-1],[s2, 1]], # Es^ + Al-Bf - Bf + Hd + 1/2 En - Di (FaTs) \
    [[r1,  0],[r2,-1],[r3, 1],[r4, 1],[r5, 0],[r6,  0],[r7,  0],[s1, 1],[s2, 0]], # Es^ - Al-Bf + Bf (CaFs) \
    [[r1,  0],[r2,-1],[r3, 1],[r4, 1],[r5, 0],[r6,  1],[r7,  0],[s1, 1],[s2,-1]], # Es^ - Al-Bf + Bf + En - Di (MaFs) \
    [[r1,  1],[r2,-1],[r3, 1],[r4, 1],[r5, 0],[r6,1/2],[r7,  0],[s1, 1],[s2, 1]], # Es^ - Al-Bf + Bf + Hd + 1/2 En - 2 Di (FaFs) \
    [[r1,  0],[r2,-1],[r3, 1],[r4, 0],[r5, 1],[r6,  0],[r7,  0],[s1, 1],[s2, 0]], # Jd + Bf - Al-Bf (Jd with Fe3+) \
    [[r1,  0],[r2,-1],[r3, 1],[r4,-1],[r5, 1],[r6,  0],[r7,  1],[s1, 0],[s2, 0]], # Jd + Cr + Bf - Es^ - Al-Bf (Jd with Fe3+) \
    [[r1,  0],[r2, 1],[r3, 1],[r4,-1],[r5, 1],[r6,  0],[r7,  0],[s1, 0],[s2, 0]], # Jd + Al-Bf + Bf - Es - Di (NaTiAlSiO6) \
    [[r1,  0],[r2, 0],[r3, 2],[r4,-1],[r5, 1],[r6,  0],[r7,  0],[s1, 0],[s2, 0]], # Jd + 2 Bf - Es - Di (NaTiFeSiO6) \
    [[r1,  0],[r2, 0],[r3, 1],[r4,-1],[r5, 1],[r6,  0],[r7,  0],[s1, 0],[s2, 0]], # Jd + Bf - Es (NaMg1/2Ti1/2Si2O6) \
    [[r1,1/2],[r2, 0],[r3, 1],[r4,-1],[r5, 1],[r6,  0],[r7,  0],[s1, 0],[s2, 0]], # Jd + Bf - Es + 1/2 Hd - 1/2 Di (NaFe1/2Ti1/2Si2O6) \
    [[r1,  2],[r2, 0],[r3, 0],[r4, 0],[r5, 0],[r6,  1],[r7,  0],[s1, 0],[s2, 1]], # 2 Hd + En - 2 Di (Fs) \
    [[r1,  1],[r2, 0],[r3, 0],[r4, 0],[r5, 0],[r6,1/2],[r7,  0],[s1, 0],[s2,-1]], # Hd + 1/2 En - Di (MgFe) \
    [[r1,  1],[r2, 0],[r3, 0],[r4, 0],[r5, 0],[r6,1/2],[r7,  0],[s1, 0],[s2, 1]], # Hd + 1/2 En - Di (FeMg) \
    [[r1,  0],[r2, 0],[r3, 0],[r4, 0],[r5, 0],[r6,  1],[r7,  1],[s1, 0],[s2,-1]], # Cr + En - Di (MaCr) \
    [[r1,  1],[r2, 0],[r3, 0],[r4, 0],[r5, 0],[r6,  1],[r7,  1],[s1, 0],[s2, 1]], # Cr + Hd + En - 2Di (FaCr) \
    [[r1,  0],[r2,-1],[r3, 1],[r4, 0],[r5, 0],[r6,  0],[r7,  1],[s1, 0],[s2, 0]], # Cr + Bf - Al-Bf  (CaCrFe3+) \
    [[r1,  0],[r2,-1],[r3, 1],[r4, 0],[r5, 0],[r6,  1],[r7,  1],[s1, 0],[s2,-1]], # Cr + Bf + En - Al-Bf - Di  (MaCrFe3+) \
    [[r1,  1],[r2,-1],[r3, 1],[r4, 0],[r5, 0],[r6,  1],[r7,  1],[s1, 0],[s2, 1]]  # Cr + Bf + Hd + En - Al-Bf - 2 Di  (FaCrFe3+) \
]

## Define standard state properties of endmember components and species

In [ ]:
ss_list = []
for i in range(1,nc+1):
    ss_string = 'mu' + str(i)
    ss_list.append(sym.Function(ss_string)(T,P))
mu0 = sym.Matrix(ss_list)
(Di,Hd,Al_Bf,Bf,Es,Jd,En,Cr) = (0,1,2,3,4,5,6,7)

In [ ]:
Delta_3A,Delta_3B,Delta_3C,Delta_3D,Delta_3E = sym.symbols('Delta_3A Delta_3B Delta_3C Delta_3D Delta_3E')
Delta_4A,Delta_4B,Delta_4C,Delta_4D,Delta_4E = sym.symbols('Delta_4A Delta_4B Delta_4C Delta_4D Delta_4E')
Delta_5A,Delta_5B,Delta_5C,Delta_5D,Delta_5E = sym.symbols('Delta_5A Delta_5B Delta_5C Delta_5D Delta_5E')
Delta_5F,Delta_5G,Delta_5H,Delta_5I,Delta_5J,Delta_5K = sym.symbols('Delta_5F Delta_5G Delta_5H Delta_5I Delta_5J Delta_5K')
Delta_6A,Delta_6B,Delta_6C,Delta_6D,Delta_6E,Delta_6F = sym.symbols('Delta_6A Delta_6B Delta_6C Delta_6D Delta_6E Delta_6F')
Delta_7A,Delta_7B,Delta_7C = sym.symbols('Delta_7A Delta_7B Delta_7C')
Delta_8A,Delta_8B,Delta_8C,Delta_8D,Delta_8E = sym.symbols('Delta_8A Delta_8B Delta_8C Delta_8D Delta_8E')
species_mu0 = [ \
    Delta_3A + mu0[Al_Bf]+mu0[En]-mu0[Di],                             # Al-Bf + En - Di \
    Delta_3B + mu0[Al_Bf]+mu0[Hd]+mu0[En]/2-2*mu0[Di],                 # Al-Bf + Hd + 1/2 En - 2 Di \
    Delta_3C + mu0[Al_Bf]+mu0[Hd]/2-mu0[Di]/2,                         # Al-Bf + 1/2 Hd - 1/2 Di \
    Delta_3D + mu0[Al_Bf]+mu0[En]+mu0[Hd]/2-3*mu0[Di]/2,               # Al-Bf + En + 1/2 Hd - 3/2 Di \
    Delta_3E + mu0[Al_Bf]+mu0[En]+3*mu0[Hd]/2-5*mu0[Di]/2,             # Al-Bf + 3/2 Hd + En - 5/2 Di \
    Delta_4A + mu0[Bf]+mu0[En]-mu0[Di],                                # Bf + En - Di \
    Delta_4B + mu0[Bf]+mu0[Hd]+mu0[En]/2-2*mu0[Di],                    # Bf + Hd + 1/2 En - 2 Di \
    Delta_4C + mu0[Bf]+mu0[Hd]/2-mu0[Di]/2,                            # Bf + 1/2 Hd - 1/2 Di \
    Delta_4D + mu0[Bf]+mu0[En]+mu0[Hd]/2-3*mu0[Di]/2,                  # Bf + En + 1/2 Hd - 3/2 Di \
    Delta_4E + mu0[Bf]+mu0[En]+3*mu0[Hd]/2-5*mu0[Di]/2,                # Bf + 3/2 Hd + En - 5/2 Di \
    Delta_5A + mu0[Es]+mu0[En]-mu0[Di],                                # Es + En - Di \
    Delta_5B + mu0[Es]+mu0[Hd]+mu0[En]/2-2*mu0[Di],                    # Es + Hd + 1/2 En - 2 Di \
    Delta_5C + mu0[Es],                                                # Es^ \
    Delta_5D + mu0[Es]+mu0[En]-mu0[Di],                                # Es^ + En - Di \
    Delta_5E + mu0[Es]+mu0[Hd]+mu0[En]/2-2*mu0[Di],                    # Es^ + Hd + 1/2 En - 2 Di \
    Delta_5F + mu0[Es]+mu0[Al_Bf]-mu0[Bf],                             # Es^ + Al-Bf - Bf (CaTs) \
    Delta_5G + mu0[Es]+mu0[Al_Bf]-mu0[Bf]+mu0[En]-mu0[Di],             # Es^ + Al-Bf - Bf + En - Di (MaTs) \
    Delta_5H + mu0[Es]+mu0[Al_Bf]-mu0[Bf]+mu0[Hd]+mu0[En]/2-mu0[Di],   # Es^ + Al-Bf - Bf + Hd + 1/2 En - Di (FaTs) \
    Delta_5I + mu0[Es]-mu0[Al_Bf]+mu0[Bf],                             # Es^ - Al-Bf + Bf (CaFs) \
    Delta_5J + mu0[Es]-mu0[Al_Bf]+mu0[Bf]+mu0[En]-mu0[Di],             # Es^ - Al-Bf + Bf + En - Di (MaFs) \
    Delta_5K + mu0[Es]-mu0[Al_Bf]+mu0[Bf]+mu0[Hd]+mu0[En]/2-2*mu0[Di], # Es^ - Al-Bf + Bf + Hd + 1/2 En - 2 Di (FaFs) \
    Delta_6A + mu0[Jd]-mu0[Al_Bf]+mu0[Bf],                             # Jd + Bf - Al-Bf (Jd with Fe3+) \
    Delta_6B + mu0[Jd]+mu0[Cr]+mu0[Bf]-mu0[Al_Bf]-mu0[Es],             # Jd + Cr + Bf - Es^ - Al-Bf (Jd with Fe3+) \
    Delta_6C + mu0[Jd]+mu0[Al_Bf]+mu0[Bf]-mu0[Es]-mu0[Di],             # Jd + Al-Bf + Bf - Es - Di (NaTiAlSiO6) \
    Delta_6D + mu0[Jd]+2*mu0[Bf]-mu0[Es]-mu0[Di],                      # Jd + 2 Bf - Es - Di (NaTiFeSiO6) \
    Delta_6E + mu0[Jd]+mu0[Bf]-mu0[Es],                                # Jd + Bf - Es (NaMg1/2Ti1/2Si2O6) \
    Delta_6F + mu0[Jd]+mu0[Bf]-mu0[Es]+mu0[Hd]/2-mu0[Di]/2,            # Jd + Bf - Es + 1/2 Hd - 1/2 Di (NaFe1/2Ti1/2Si2O6) \
    Delta_7A + 2*mu0[Hd]+mu0[En]-2*mu0[Di],                            # 2 Hd + En - 2 Di (Fs) \
    Delta_7B + mu0[Hd]+mu0[En]/2-mu0[Di],                              # Hd + 1/2 En - Di (Fs) \
    Delta_7C + mu0[Hd]+mu0[En]/2-mu0[Di],                              # Hd + 1/2 En - Di (Fs) \
    Delta_8A + mu0[Cr]+mu0[En]-mu0[Di],                                # Cr + En - Di (MaCr) \
    Delta_8B + mu0[Cr]+mu0[Hd]+mu0[En]-2*mu0[Di],                      # Cr + Hd + En - 2Di (FaCr) \
    Delta_8C + mu0[Cr]+mu0[Bf]-mu0[Al_Bf],                             # Cr + Bf - Al-Bf  (CaCrFe3+) \
    Delta_8D + mu0[Cr]+mu0[Bf]+mu0[En]-mu0[Al_Bf]-mu0[Di],             # Cr + Bf + En - Al-Bf - Di  (MaCrFe3+) \
    Delta_8E + mu0[Cr]+mu0[Bf]+mu0[Hd]+mu0[En]-mu0[Al_Bf]-2*mu0[Di]    # Cr + Bf + Hd + En - Al-Bf - 2 Di  (FaCrFe3+) \
]

## Construct a matrix to map Taylor expansion coefficients to physical variables 
The variable *taylor_m* will contain a list of lists of rows; each row is of length 55, equal to the number of Taylor expansion coefficients. The first 8 rows are defined, one for each standard state component property.  The varible *taylor_b* hold the result, in this case the standard state chemical potential. The coefficients in the matrix are the multipliers of the Taylor expansion coefficients.

In [ ]:
count = 0
taylor_m = []
taylor_b = []
for y in endmembers[:]:
    row = [sym.S.One]
    for x in taylor_terms[1:]:
        row.append(x.subs(y))
    taylor_m.append(row)
    taylor_b.append(mu0[count])
    count += 1

In [ ]:
#sym.Matrix(taylor_m)

In [ ]:
#taylor_b

Add species terms

In [ ]:
count = 0
for y in species[:]:
    row = [sym.S.One]
    for x in taylor_terms[1:]:
        row.append(x.subs(y))
    taylor_m.append(row)
    taylor_b.append(species_mu0[count])
    count += 1

In [ ]:
#sym.Matrix(taylor_m)

In [ ]:
#taylor_b

Evaluate the rank and nullspace of this matrix

In [ ]:
print ('There are {0} rows in the endmembers+species constraint matrix'.format(len(taylor_b)))
G_range_space_matrix = sym.Matrix(taylor_m)
G_null_space_matrix = G_range_space_matrix.nullspace()
ns_rank = len(G_null_space_matrix)
print ('There are {0} columns in the null space of Taylor expansion coefficients for the intra-site parameters.'.format(ns_rank))
print ('The null space vectors are:')
for i in range(0,ns_rank):
    constraint = sym.S.Zero
    col = G_null_space_matrix[i]
    for j in range(0,len(col)):
        if col[j] != 0:
            constraint += sym.nsimplify(col[j])*taylor_coeff[j]
    display(constraint)

## Next, construct the binary interaction terms
#### There are a number of ways to do this step ...
Number of endmembers is the number of components + the number of ordering parameters, since some components can exist in multiple ordering states, e.g. ${\rm{CaFeAlSi}}{{\rm{O}}_{\rm{6}}}$ and ${\rm{CaAlFeSi}}{{\rm{O}}_{\rm{6}}}$ for $s_1$ and ${\rm{FeMgS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$ and ${\rm{MgFeS}}{{\rm{i}}_{\rm{2}}}{{\rm{O}}_{\rm{6}}}$ for $s_2$

In [ ]:
ne = nc + ns
print ('There are {0:.0f} stndard state properties.'.format(ne))

In [ ]:
nt = 1 + nc - 1 + ns + (nc-1)*(nc-2)/2 + ns*(ns-1)/2 + ns*(nc-1) + nc-1 + ns
print ('Number of Taylor expansion coefficients for a second order expansion is {0:.0f}.'.format(nt))

#### One method is to define strictly regular solution terms (including interactions with ordering endmembers):

In [ ]:
nw = ne*(ne-1)/2
print ('For a strictly regular solution this yields {0:.0f} cofficients + {1:.0f} compositional/ordering endmembers.'.format(nw, ne))

#### A better method is to isolate intra-site interactions from inter-site interactions ...
## Intra-site terms

It is useful to define interaction terms isolated to a per site basis.  
We have 4 M2 sites: $X_{Na}^{M2}$, $X_{Ca}^{M2}$, $X_{Mg}^{M2}$, $X_{{Fe}^{2+}}^{M2}$  
We have 6 M1 sites: $X_{Mg}^{M1}$, $X_{{Fe}^{2+}}^{M1}$, $X_{Al}^{M1}$, $X_{Cr}^{M1}$, $X_{{Fe}^{3+}}^{M1}$, $X_{Ti}^{M1}$  
We have 3 tetrahedral sites: $X_{Al}^{Tet}$, $X_{{Fe}^{3+}}^{Tet}$, $X_{Si}^{Tet}$  

In [ ]:
ni = 4*(4-1)/2 + 6*(6-1)/2 + 3*(3-1)/2
print ('So, the total number of intrasite W interaction parameters is: {0:.0f}'.format(ni))
print ('And the number of cross-site interaction terms is: {0:.0f}'.format(nw-ni))

### Intra-site Parameter symbols
Regular solution interaction terms between cations on each site

In [ ]:
wM2_Na_Ca, wM2_Na_Mg, wM2_Na_Fe2, wM2_Ca_Mg, wM2_Ca_Fe2, wM2_Mg_Fe2 \
= sym.symbols('wM2_Na_Ca wM2_Na_Mg wM2_Na_Fe2 wM2_Ca_Mg wM2_Ca_Fe2 wM2_Mg_Fe2')
wM1_Mg_Fe2, wM1_Mg_Al, wM1_Mg_Cr, wM1_Mg_Fe3, wM1_Mg_Ti, wM1_Fe2_Al, wM1_Fe2_Cr, wM1_Fe2_Fe3, wM1_Fe2_Ti, \
wM1_Al_Cr, wM1_Al_Fe3, wM1_Al_Ti, wM1_Cr_Fe3, wM1_Cr_Ti, wM1_Fe3_Ti \
= sym.symbols('wM1_Mg_Fe2 wM1_Mg_Al wM1_Mg_Cr wM1_Mg_Fe3 wM1_Mg_Ti wM1_Fe2_Al wM1_Fe2_Cr wM1_Fe2_Fe3 wM1_Fe_Ti \
wM1_Al_Cr wM1_Al_Fe3 wM1_Al_Ti wM1_Cr_Fe3 wM1_Cr_Ti wM1_Fe3_Ti')
wT_Al_Fe3, wT_Al_Si, wT_Fe3_Si = sym.symbols('wT_Al_Fe3 wT_Al_Si wT_Fe3_Si')

### Intra-site Terms and Energetic contributions

In [ ]:
intra_site_terms = [[xM2_Na,xM2_Ca], [xM2_Na,xM2_Mg], [xM2_Na,xM2_Fe2], [xM2_Ca,xM2_Mg], [xM2_Ca,xM2_Fe2], [xM2_Mg,xM2_Fe2], \
                    [xM1_Mg,xM1_Fe2], [xM1_Mg,xM1_Al], [xM1_Mg,xM1_Cr], [xM1_Mg,xM1_Fe3], [xM1_Mg,xM1_Ti], [xM1_Fe2,xM1_Al], [xM1_Fe2,xM1_Cr], [xM1_Fe2,xM1_Fe3], \
                    [xM1_Fe2,xM1_Ti], [xM1_Al,xM1_Cr], [xM1_Al,xM1_Fe3], [xM1_Al,xM1_Ti], [xM1_Cr,xM1_Fe3],  [xM1_Cr,xM1_Ti], [xM1_Fe3,xM1_Ti], \
                    [xT_Al,xT_Fe3], [xT_Al,xT_Si], [xT_Fe3,xT_Si]]
intra_site_coeff = [wM2_Na_Ca, wM2_Na_Mg, wM2_Na_Fe2, wM2_Ca_Mg, wM2_Ca_Fe2, wM2_Mg_Fe2, wM1_Mg_Fe2, wM1_Mg_Al, \
                    wM1_Mg_Cr, wM1_Mg_Fe3, wM1_Mg_Ti, wM1_Fe2_Al, wM1_Fe2_Cr, wM1_Fe2_Fe3, wM1_Fe2_Ti, wM1_Al_Cr, \
                    wM1_Al_Fe3, wM1_Al_Ti, wM1_Cr_Fe3, wM1_Cr_Ti, wM1_Fe3_Ti, wT_Al_Fe3, wT_Al_Si, wT_Fe3_Si]
w = replace
intra_site_defns = [[xM2_Na.subs(w)-1/2, xM2_Ca.subs(w)-1/2, xM2_Mg.subs(w),     xM2_Fe2.subs(w)    ],
                    [xM2_Na.subs(w)-1/2, xM2_Ca.subs(w),     xM2_Mg.subs(w)-1/2, xM2_Fe2.subs(w)    ],
                    [xM2_Na.subs(w)-1/2, xM2_Ca.subs(w),     xM2_Mg.subs(w),     xM2_Fe2.subs(w)-1/2],
                    [xM2_Na.subs(w),     xM2_Ca.subs(w)-1/2, xM2_Mg.subs(w)-1/2, xM2_Fe2.subs(w)    ],
                    [xM2_Na.subs(w),     xM2_Ca.subs(w)-1/2, xM2_Mg.subs(w),     xM2_Fe2.subs(w)-1/2],
                    [xM2_Na.subs(w),     xM2_Ca.subs(w),     xM2_Mg.subs(w)-1/2, xM2_Fe2.subs(w)-1/2],
                    [xM1_Mg.subs(w)-1/2, xM1_Fe2.subs(w)-1/2, xM1_Al.subs(w),     xM1_Cr.subs(w),     xM1_Fe3.subs(w),     xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)-1/2, xM1_Fe2.subs(w),     xM1_Al.subs(w)-1/2, xM1_Cr.subs(w),     xM1_Fe3.subs(w),     xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)-1/2, xM1_Fe2.subs(w),     xM1_Al.subs(w),     xM1_Cr.subs(w)-1/2, xM1_Fe3.subs(w),     xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)-1/2, xM1_Fe2.subs(w),     xM1_Al.subs(w),     xM1_Cr.subs(w),     xM1_Fe3.subs(w)-1/2, xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)-1/2, xM1_Fe2.subs(w),     xM1_Al.subs(w),     xM1_Cr.subs(w),     xM1_Fe3.subs(w),     xM1_Ti.subs(w)-1/2],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)-1/2, xM1_Al.subs(w)-1/2, xM1_Cr.subs(w),     xM1_Fe3.subs(w),     xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)-1/2, xM1_Al.subs(w),     xM1_Cr.subs(w)-1/2, xM1_Fe3.subs(w),     xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)-1/2, xM1_Al.subs(w),     xM1_Cr.subs(w),     xM1_Fe3.subs(w)-1/2, xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)-1/2, xM1_Al.subs(w),     xM1_Cr.subs(w),     xM1_Fe3.subs(w),     xM1_Ti.subs(w)-1/2],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)    , xM1_Al.subs(w)-1/2, xM1_Cr.subs(w)-1/2, xM1_Fe3.subs(w),     xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)    , xM1_Al.subs(w)-1/2, xM1_Cr.subs(w),     xM1_Fe3.subs(w)-1/2, xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)    , xM1_Al.subs(w)-1/2, xM1_Cr.subs(w),     xM1_Fe3.subs(w),     xM1_Ti.subs(w)-1/2],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)    , xM1_Al.subs(w),     xM1_Cr.subs(w)-1/2, xM1_Fe3.subs(w)-1/2, xM1_Ti.subs(w)    ],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)    , xM1_Al.subs(w),     xM1_Cr.subs(w)-1/2, xM1_Fe3.subs(w),     xM1_Ti.subs(w)-1/2],
                    [xM1_Mg.subs(w)    , xM1_Fe2.subs(w)    , xM1_Al.subs(w),     xM1_Cr.subs(w),     xM1_Fe3.subs(w)-1/2, xM1_Ti.subs(w)-1/2],
                    [xT_Al.subs(w)-1/4, xT_Fe3.subs(w)-1/4, xT_Si.subs(w)-1/2],
                    [xT_Al.subs(w)-1/4, xT_Fe3.subs(w),     xT_Si.subs(w)-3/4],
                    [xT_Al.subs(w),     xT_Fe3.subs(w)-1/4, xT_Si.subs(w)-3/4]
                   ]
print (len(intra_site_terms), len(intra_site_coeff))

In [ ]:
for x in intra_site_defns[21:]:
    x.append(r1)
    x.append(r5)
    x.append(r6)
    display(sym.linsolve(x,r1,r2,r3,r4,r5,r6,r7,s1,s2).args[0])

### Add intrasite terms to the matrix that maps Taylor expansion coefficients to physical variables

In [ ]:
count = 0
for x in intra_site_terms[:]:
    print(x.subs(replace))
    p = sym.Poly(x.subs(replace), r1, r2, r3, r4, r5, r6, r7, s1, s2)
    row = [p.coeff_monomial(sym.S.One)]
    for i in range(1,nc):
        row.append(p.coeff_monomial(var[i-1]))
    for i in range(1,ns+1):
        row.append(p.coeff_monomial(var[nc-2+i]))
    for i in range(1,nc):
        for j in range(i,nc):
            row.append(p.coeff_monomial(var[i-1]*var[j-1]))
        for j in range(1,ns+1):
            row.append(p.coeff_monomial(var[i-1]*var[nc-2+j]))
    for i in range(1,ns+1):
        for j in range(i,ns+1):
            row.append(p.coeff_monomial(var[nc-2+i]*var[nc-2+j]))
    taylor_m.append(row)
    taylor_b.append(intra_site_coeff[count])
    count += 1
sym.Matrix(taylor_m)

In [ ]:
sym.Matrix(taylor_b)

### Total intra-site non-configurational Gibbs Free energy

In [ ]:
G_intra_site = sym.S.Zero
count = 0
for x in intra_site_terms[:]:
    G_intra_site += intra_site_coeff[count]*x[0]*x[1]
    count += 1
G_intra_site

#### Intra-site Gibbs energies for the endmembers

In [ ]:
print ('Di, Hd, Al-Bf, Bf, Es, Jd, En, Cr')
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,0],[r2,1],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,1],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,0],[r4,1],[r5,0],[r6,0],[r7,0],[s1, 1],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,1],[r6,0],[r7,0],[s1,-1],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2,-1]]))
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,1],[s1, 0],[s2, 0]]))
print ('anti-Es, Fs, FeMgSi2O6, MgFeSi2O6')
display(G_intra_site.subs(replace).subs([[r1,0],[r2,0],[r3,0],[r4,1],[r5,0],[r6,0],[r7,0],[s1,-1],[s2, 0]]))
display(G_intra_site.subs(replace).subs([[r1,2],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2, 1]]))
display(G_intra_site.subs(replace).subs([[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2, 1]]))
display(G_intra_site.subs(replace).subs([[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2,-1]]))

## Inter-site non-configurational Gibbs Free energy
This function is determined by finding the nullspace of Taylor expansion coefficients that are constrained by the intra-site interactions and the standard state terms. The dimension of this null space is already known.  

The range space of the matrix of constraints is defined first. We then determine the null space of that matrix, its dimension, and assign symbol names to each of the inter-site (null space) interaction parameters.

In [ ]:
G_range_space_matrix = sym.Matrix(taylor_m)
G_null_space_matrix = G_range_space_matrix.nullspace()
print ('There are {0} columns in the null space of Taylor expansion coefficients for the intra-site parameters.'.format(len(G_null_space_matrix)))
wInter = []
for i in range(0,len(G_null_space_matrix)):
    wInter.append(sym.symbols('wInter_'+str(i+1)))
wInter

In [ ]:
print ('Null-space matrix has {0} columns, each with {1} rows.'.format(len(G_null_space_matrix), len(G_null_space_matrix[0])))

Finally, we construct from the null space matrix an expession for the inter-site non-configurational Gibbs free energy.

In [ ]:
G_inter_site = sym.S.Zero
entry = 0
for col in G_null_space_matrix[:]:
    defn = sym.S.Zero
    count = 0
    if col[count] != 0:
        defn += col[count]
    count += 1
    for i in range(1,nc):
        if col[count] != 0:
            defn += col[count]*var[i-1]
        count += 1
    for i in range(1,ns+1):
        if col[count] != 0:
            defn += col[count]*var[nc-2+i]
        count += 1
    for i in range(1,nc):
        for j in range(i,nc):
            if col[count] != 0:
                defn += col[count]*var[i-1]*var[j-1]
            count += 1
        for j in range(1,ns+1):
            if col[count] != 0:
                defn += col[count]*var[i-1]*var[nc-2+j]
            count += 1
    for i in range(1,ns+1):
        for j in range(i,ns+1):
            if col[count] != 0:
                defn += col[count]*var[nc-2+i]*var[nc-2+j]
            count += 1
    G_inter_site += defn*wInter[entry]
    entry += 1
G_inter_site

## $G^*$

In [ ]:
Gstar = mu0[0]*(1-r1-r2-r3-r4-r5-r6-r7)+mu0[1]*r1+mu0[2]*r2+mu0[3]*r3+mu0[4]*r4+mu0[5]*r5+mu0[6]*r6+mu0[7]*r7 + G_intra_site.subs(replace) + G_inter_site
Gstar

#### Inter-site Gibbs energies of the endmembers

In [ ]:
print ('Di, Hd, Al-Bf, Bf, Es, Jd, En, Cr')
display(Gstar.subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(Gstar.subs([[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(Gstar.subs([[r1,0],[r2,1],[r3,0],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(Gstar.subs([[r1,0],[r2,0],[r3,1],[r4,0],[r5,0],[r6,0],[r7,0],[s1, 0],[s2, 0]]))
display(Gstar.subs([[r1,0],[r2,0],[r3,0],[r4,1],[r5,0],[r6,0],[r7,0],[s1, 1],[s2, 0]]))
display(Gstar.subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,1],[r6,0],[r7,0],[s1,-1],[s2, 0]]))
display(Gstar.subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2,-1]]))
display(Gstar.subs([[r1,0],[r2,0],[r3,0],[r4,0],[r5,0],[r6,0],[r7,1],[s1, 0],[s2, 0]]))
print ('anti-Es, Fs, FeMgSi2O6, MgFeSi2O6')
display(Gstar.subs([[r1,0],[r2,0],[r3,0],[r4,1],[r5,0],[r6,0],[r7,0],[s1,-1],[s2, 0]]))
display(Gstar.subs([[r1,2],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2, 1]]))
display(Gstar.subs([[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2, 1]]))
display(Gstar.subs([[r1,1],[r2,0],[r3,0],[r4,0],[r5,0],[r6,1],[r7,0],[s1, 0],[s2,-1]]))

In [ ]:
sub_for_rs = [ \
    [r1, xM2_Fe2 + xM1_Fe2], \
    [r2, (xM1_Al-xM1_Fe3-xM2_Na)/2 + (xT_Al-xT_Fe3-xM1_Cr/2) + xM1_Ti], \
    [r3, (xM1_Fe3-xM1_Al+xM2_Na)/2 + (xT_Fe3-xT_Al+xM1_Cr/2) + xM1_Ti], \
    [r4, (xM1_Al+xM1_Fe3-xM2_Na)/2 + (xT_Al+xT_Fe3-xM1_Cr/2) - xM1_Ti], \
    [r5, xM2_Na], \
    [r6, 1 - xM2_Na - xM2_Ca], \
    [r7, xM1_Cr], \
    [s1, xM1_Fe3 - xM1_Al], \
    [s2, xM2_Fe2 - xM2_Mg] \
]
#inter_site_gstar

## Parameterize the excess free energy
- Symmetric terms: $W_{ij} = Wh_{ij} - T Ws_{ij} + P Wv_{ij}$, where $Wh_{ij}$ is the excess enthalpy along the $i$-$j$ binary, $Ws_{ij}$ is the excess entropy, and $Wv_{ij}$ is the excess volume
- Asymetric terms: $\Delta W_{ij} = \Delta Wh_{ij} - T \Delta Ws_{ij} + P \Delta Wv_{ij}$
- Convention: $\left[ {{W_{ij}} + \Delta {W_{ij}}\left( {{X_i} - {X_j}} \right)} \right]{X_i}{X_j} = \left( {{W_{ij}} + \Delta {W_{ij}}} \right)X_i^2{X_j} + \left( {{W_{ij}} - \Delta {W_{ij}}} \right){X_i}X_j^2 = {W_{iij}}X_i^2{X_j} + {W_{ijj}}{X_i}X_j^2$  
- Strictly ternary terms: $W_{ijk}X_iX_jX_k$, where $i {\ne} j {\ne} k$

In [ ]:
module = 'asymm_regular'
params = []
units = []
symparam = ()
G_excess = sym.symbols('G_excess')
G_excess = 0
for i in range(1,c):
    for j in range(i+1,c+1):
        param = 'Wh' + str(i) + str(j); params.append(param); units.append('J/m')
        h_term = sym.symbols(param); symparam += (h_term,)
        param = 'Ws' + str(i) + str(j); params.append(param); units.append('J/K-m')
        s_term = sym.symbols(param); symparam += (s_term,)
        param = 'Wv' + str(i) + str(j); params.append(param); units.append('J/bar-m')
        v_term = sym.symbols(param); symparam += (v_term,)
        param = 'dWh' + str(i) + str(j); params.append(param); units.append('J/m')
        dh_term = sym.symbols(param); symparam += (dh_term,)
        param = 'dWs' + str(i) + str(j); params.append(param); units.append('J/K-m')
        ds_term = sym.symbols(param); symparam += (ds_term,)
        param = 'dWv' + str(i) + str(j); params.append(param); units.append('J/bar-m')
        dv_term = sym.symbols(param); symparam += (dv_term,)
        w_term = h_term - T*s_term + P*v_term
        dw_term = dh_term - T*ds_term + P*dv_term
        G_excess += (w_term + dw_term*(n[i-1]-n[j-1])/nT)*n[i-1]*n[j-1]
G_excess /= nT
for i in range(1,c-1):
    for j in range(i+1,c):
        for k in range(j+1,c+1):
            param = 'Wh' + str(i) + str(j) + str(k); params.append(param); units.append('J/m')
            h_term = sym.symbols(param); symparam += (h_term,)
            param = 'Ws' + str(i) + str(j) + str(k); params.append(param); units.append('J/K-m')
            s_term = sym.symbols(param); symparam += (s_term,)
            param = 'Wv' + str(i) + str(j) + str(k); params.append(param); units.append('J/bar-m')
            v_term = sym.symbols(param); symparam += (v_term,)
            G_excess += (h_term - T*s_term + P*v_term)*n[i-1]*n[j-1]*n[k-1]/nT/nT
G_excess

In [ ]:
print(params)
print(units)

## Define the Gibbs free energy of solution

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

In [ ]:
G = G_ss + G_config + G_excess
G